## Init

In [95]:
import sys
# print(sys.path)

!python --version

!pip install -q mediapipe

# 모델 파일 다운로드 (Pose Landmarker Lite, Float16, v1)
!wget -O pose_landmarker.task -q https://storage.googleapis.com/mediapipe-models/pose_landmarker/pose_landmarker_heavy/float16/1/pose_landmarker_heavy.task

Python 3.12.7


'wget'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.


권장 버전

- Python 3.12.7
    - 3.10 ~ 3.12
- mediapipe: 0.10.21
- numpy: 1.26.4

## 실시간으로 MPP 동작

### 실시간 - calib 없음

여기서부터 아래 전체 실행하기

변경 가능사항

In [ ]:
# 내장 카메라 or 외장 USB 웹캠 선택
# CAM_INDEX = 0  # 내장 카메라
CAM_INDEX = 1 # 외장 USB 웹캠
# 영상 너비 결정
MAX_WIDTH = 650
# 목표 스쿼트 횟수(현재 3회 완료 시 자동 정지 후 결과 출력)
TARGET_REPS = 3

In [256]:
import math
import time
import cv2
import numpy as np
import mediapipe as mp

# MediaPipe 초기화
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

# 스쿼트 영상 경로
# CAM_INDEX = 0  # 내장 카메라
# CAM_INDEX = 1 # 외장 USB 웹캠
USE_DIRECTSHOW = True
RESTART_RESET = False
# squat_videoPath = 'C:\\Users\\mia00\\Desktop\\CD\\resources\\squat.mp4'
squat_videoPath = "C:\\Users\\mia00\\Desktop\\CD\\resources\\2.mp4"

# MAX_WIDTH = 1000  # 영상 최대 너비
FEEDBACK_COLOR = (255, 255, 255)
EXTRA_COLOR    = (0, 170, 255)

TIME_LIMIT = 10  # 시간 제한 (초)
SIDE_VIS_MARGIN = 0.15          # visibility 합 차이 최소 마진
SIDE_STABILITY_FRAMES = 10      # 연속 프레임 유지 필요 수
MIN_VIS = 0.7 # 최소 visibility
# SESSION_MAX_SECONDS = 30 # 30초
session_running = False
session_start_time = None
last_display_frame = None
# TARGET_REPS = 3 # 목표 스쿼트 반복 횟수
TARGET_REP_HOLD_FRAMES = 3 # 완료 후 유지 프레임 수
COUNTDOWN_SECONDS = 3 # 시작 카운트다운

# MIN standard
CALIB_MIN_VIS = 0.55
BACK_STRAIGHTNESS_THRESHOLD_CALIB = 0.18
VIS_EWMA_ALPHA = 0.4
vis_ewma = {
    "left": {"HIP": None, "KNEE": None, "SHOULDER": None},
    "right": {"HIP": None, "KNEE": None, "SHOULDER": None}
}

BACK_STRAIGHTNESS_THRESHOLD = 0.15  # 허리 펴짐 기준
KNEE_MIN_OK = 70
KNEE_MAX_OK = 110
# BACK_DEV_TOL = 0.18
BACK_DEV_TOL = 0.40
IDEAL_DEPTH_RATIO = 1.0
TARGET_SIT_FRAMES = 6
W_KNEE = 0.35
W_BACK = 0.25
W_DEPTH = 0.25
W_CTRL = 0.15
# MIN_DEPTH_DELTA = 0.08
# MIN_SIT_FRAMES_FOR_DEPTH = 5      # SIT에서 최소 유지 프레임
# UP_FRAMES_CONFIRM = 3
# HIP_ABOVE_KNEE_GAP = 0.05
# INIT_HIP_MARGIN = 0.05
# VEL_WINDOW = 5
# MIN_UP_VEL = 0.003

BACK_USE_TRUNK_ANGLE = True
TRUNK_GOOD_EXTRA = 5.0
TRUNK_BAD_EXTRA  = 20.0
TRUNK_EMA_ALPHA  = 0.25

rep_scores = []
rep_grades = []
rep_breakdowns = []
recent_rep_feedback = ""
avg_score = 0.0

ENTER_SIT_GAP = 0.06
DEPTH_MIN = 0.07
BOTTOM_MIN_FRAMES = 4
VEL_WINDOW = 4
UP_VEL_THRESH = -0.0025
DOWN_VEL_THRESH = 0.0015
UP_MIN_FRAMES = 10 # up
KNEE_GAP = 0.07 # up
STAND_HIP_MARGIN = 0.03 # down
GRACE_FRAMES = 5
SLOW_VEL = -0.0005
MIN_UP_DISP = 0.04
STAND_PROGRESS = 0.90

calib_good_frames = 0
calib_hip_samples = []
calibrated = False

track = {
    "hip_init": None, # 초기 힙 위치
    "hip_bottom": None, # 가장 낮은 힙 위치
    "knee_bottom": None, # 가장 낮은 무릎 위치
    "sit_frames": 0, # SIT 프레임 수
    "bottom_locked": False, # SIT에서 바닥 확인
    "rise_frames": 0, # 상승 프레임 수
    "hip_history": [], # 최근 힙 위치 기록
    "knee_angle_min": None,
    "back_dev_max": 0.0,
    "scored": False
}
track.setdefault("trunk_angle_baseline", None)
track.setdefault("trunk_angle_ema", None)
track.setdefault("trunk_angle_peak", None)

SIDE_IDX = {
    "left": {
        "HIP": mp_pose.PoseLandmark.LEFT_HIP.value,
        "KNEE": mp_pose.PoseLandmark.LEFT_KNEE.value,
        "ANKLE": mp_pose.PoseLandmark.LEFT_ANKLE.value,
        "SHOULDER": mp_pose.PoseLandmark.LEFT_SHOULDER.value
    },
    "right": {
        "HIP": mp_pose.PoseLandmark.RIGHT_HIP.value,
        "KNEE": mp_pose.PoseLandmark.RIGHT_KNEE.value,
        "ANKLE": mp_pose.PoseLandmark.RIGHT_ANKLE.value,
        "SHOULDER": mp_pose.PoseLandmark.RIGHT_SHOULDER.value
    }
}

In [257]:
def joints_visible(landmarks, side, min_vis=MIN_VIS):
    ids = SIDE_IDX[side]
    # needed = ["HIP", "KNEE", "ANKLE", "SHOULDER"]
    needed = ["HIP", "KNEE", "SHOULDER"]
    vis_map = {name: landmarks[ids[name]].visibility for name in needed}
    ok = all(vis_map[n] >= min_vis for n in needed)
    return ok, vis_map

In [258]:
def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)

    ab = a - b
    bc = c - b

    cosine_angle = np.dot(ab, bc) / (np.linalg.norm(ab) * np.linalg.norm(bc))
    angle = np.arccos(np.clip(cosine_angle, -1.0, 1.0))
    return np.degrees(angle)

In [259]:
def check_side(landmarks, prev_side, stability_counter):
    l_ids = SIDE_IDX["left"]
    r_ids = SIDE_IDX["right"]
    l_sum = (landmarks[l_ids["HIP"]].visibility +
             landmarks[l_ids["KNEE"]].visibility +
             landmarks[l_ids["ANKLE"]].visibility)
    r_sum = (landmarks[r_ids["HIP"]].visibility +
             landmarks[r_ids["KNEE"]].visibility +
             landmarks[r_ids["ANKLE"]].visibility)

    candidate = prev_side
    diff = l_sum - r_sum
    if prev_side is None:
        if l_sum >= r_sum:
            candidate = "left"
        else:
            candidate = "right"
        stability_counter = 0
    else:
        if diff > SIDE_VIS_MARGIN and prev_side != "left":
            candidate = "left"
        elif diff < -SIDE_VIS_MARGIN and prev_side != "right":
            candidate = "right"
        else:
            return prev_side, stability_counter, l_sum, r_sum

        if candidate != prev_side:
            stability_counter += 1
            if stability_counter >= SIDE_STABILITY_FRAMES:
                prev_side = candidate
                stability_counter = 0
            else:
                candidate = prev_side
        else:
            stability_counter = 0

    return candidate, stability_counter, l_sum, r_sum

In [260]:
def check_knee_angle(landmarks, side):
    ids = SIDE_IDX[side]

    hip = [landmarks[ids["HIP"]].x, landmarks[ids["HIP"]].y]
    knee = [landmarks[ids["KNEE"]].x, landmarks[ids["KNEE"]].y]
    ankle = [landmarks[ids["ANKLE"]].x, landmarks[ids["ANKLE"]].y]

    # 무릎 각도 계산
    knee_angle = calculate_angle(hip, knee, ankle)

    # 피드백 생성
    if knee_angle < 90:
        feedback = f"Good! Knee angle: {knee_angle:.2f} degrees"
        # is_bent_correctly = True
    else:
        feedback = f"Bad! Knee angle: {knee_angle:.2f}degrees"
        # is_bent_correctly = False

    # return feedback, is_bent_correctly
    return feedback, knee_angle

In [261]:
def classify_grade(score: float) -> str:
    if score >= 85: return "Perfect"
    if score >= 70: return "Good"
    if score >= 55: return "Fair"
    return "Poor"

def clamp(v, lo, hi):
    return max(lo, min(hi, v))

- knee% : 무릎 각도 점수 정규화
    - 무릎을 충분히 굽혔는가
    - knee_angle : 허벅지-무릎-종아리로 이루어진 각도
- back% : 등/상체 정렬
    - 내려갔다가 올라오는 동안 상체가 앞으로 굽혀졌는지
    - back_dev : 어깨와 엉덩이 y 차이
- depth% : 엉덩이가 충분히 깊게 내려갔는가
    - depth_ratio : 무릎 높이를 100% 기준점으로 둔 상대 깊이
- ctrl% : 앉은 상태를 충분히 유지
    - sit_frames : SIT 상태로 머문 프레임 수

=> (a) 충분한 굽힘, (b) 허리 과도 굽힘 없음, (c) 과하거나 부족하지 않은 깊이, (d) 반동 없이 제어된 전환

In [262]:
def compute_trunk_angle(landmarks, side):
    ids = SIDE_IDX[side]
    hip = landmarks[ids["HIP"]]
    shoulder = landmarks[ids["SHOULDER"]]
    dx = shoulder.x - hip.x
    dy = shoulder.y - hip.y
    norm = (dx*dx + dy*dy) ** 0.5
    if norm < 1e-6:
        return 0.0
    # 수직(아래) 벡터(0,1)와의 각도 (0°=수직, 증가=전경)
    cosv = max(-1.0, min(1.0, dy / norm))
    return math.degrees(math.acos(cosv))

In [263]:
def compute_rep_score_simple(trk: dict):
    # 내려간 양, 무릎 각도, 허리 펴짐, 깊이 비율, 제어 프레임 수
    if (trk["knee_angle_min"] is None or
        trk["hip_init"] is None or
        trk["hip_bottom"] is None or
        trk["knee_bottom"] is None):
        return 0.0, {}, "Invalid"
    knee_angle = trk["knee_angle_min"]
    back_dev = trk["back_dev_max"]
    hip_init = trk["hip_init"]
    depth_raw = trk["hip_bottom"] - hip_init
    knee_span = trk["knee_bottom"] - hip_init if trk["knee_bottom"] > hip_init else 0
    depth_ratio = depth_raw / knee_span if knee_span > 1e-6 else 0.0

    knee_norm = clamp((KNEE_MAX_OK - knee_angle)/(KNEE_MAX_OK - KNEE_MIN_OK),0,1)
    # back_norm = clamp(1 - back_dev / BACK_DEV_TOL,0,1)
    if BACK_USE_TRUNK_ANGLE:
        base = trk.get("trunk_angle_baseline")
        peak = trk.get("trunk_angle_peak", base if base is not None else 0.0)
        if base is None:
            back_norm = 1.0
            back_dev_out = 0.0
        else:
            extra = max(0.0, peak - base)   # baseline 대비 증가각
            if extra <= TRUNK_GOOD_EXTRA:
                back_norm = 1.0
            elif extra >= TRUNK_BAD_EXTRA:
                back_norm = 0.0
            else:
                back_norm = 1 - (extra - TRUNK_GOOD_EXTRA)/(TRUNK_BAD_EXTRA - TRUNK_GOOD_EXTRA)
            back_dev_out = round(extra,2)
    else:
        back_dev = trk["back_dev_max"]
        back_norm = clamp(1 - back_dev / BACK_DEV_TOL,0,1)
        back_dev_out = round(back_dev,3)

    # FIX: depth dead‑zone (±0.05 이내는 만점)
    raw = abs(depth_ratio - IDEAL_DEPTH_RATIO)
    if raw < 0.05:
        depth_norm = 1.0
    else:
        depth_norm = clamp(1 - raw/IDEAL_DEPTH_RATIO, 0, 1)

    # depth_norm = clamp(1 - abs(depth_ratio - IDEAL_DEPTH_RATIO)/IDEAL_DEPTH_RATIO,0,1)
    ctrl_norm = clamp(trk["sit_frames"]/TARGET_SIT_FRAMES,0,1)
    score = 100*(W_KNEE*knee_norm + W_BACK*back_norm + W_DEPTH*depth_norm + W_CTRL*ctrl_norm)
    grade = classify_grade(score)
    breakdown = {
        "knee%": round(knee_norm*100,1),
        "back%": round(back_norm*100,1),
        "depth%": round(depth_norm*100,1),
        "ctrl%": round(ctrl_norm*100,1),
        "knee_angle": round(knee_angle,1),
        "depth_ratio": round(depth_ratio,2),
        # "back_dev": round(back_dev,3),
        "back_dev": back_dev_out,
        "sit_frames": trk["sit_frames"]
    }
    return round(score,1), breakdown, grade

In [264]:
def put_text_with_bg(img, text, org, color, font=cv2.FONT_HERSHEY_SIMPLEX,
                     scale=0.8, thickness=2, bg_color=(30,30,30), alpha=0.6, pad=6):
    (tw, th), bl = cv2.getTextSize(text, font, scale, thickness)
    x, y = org
    # 배경 영역
    bg_tl = (x - pad, y - th - pad)
    bg_br = (x + tw + pad, y + bl + pad//2)
    # 반투명 배경
    overlay = img.copy()
    cv2.rectangle(overlay, bg_tl, bg_br, bg_color, -1)
    cv2.addWeighted(overlay, alpha, img, 1 - alpha, 0, img)
    # 텍스트
    cv2.putText(img, text, (x, y), font, scale, color, thickness, cv2.LINE_AA)

In [265]:
def draw_feedback(image, feedback, state, squat_count, extra_lines=None):
    put_text_with_bg(
        image,
        f"State: {state} | Count: {squat_count}",
        (30, 50),
        (0, 255, 0),  # 상태/카운트는 녹색 유지
        scale=0.9
    )
    if feedback:
        put_text_with_bg(
            image,
            feedback,
            (30, 95),
            FEEDBACK_COLOR,
            scale=0.85
        )
    if extra_lines:
        y = 140
        for line in extra_lines:
            put_text_with_bg(
                image,
                line,
                (30, y),
                EXTRA_COLOR,
                scale=0.65
            )
            y += 38

In [266]:
def resize_image(image, max_width):
    h, w = image.shape[:2]
    if w == max_width:
        return image
    scale = max_width / w
    return cv2.resize(image, (max_width, int(h * scale)), interpolation=cv2.INTER_LINEAR)

In [267]:
# def process_state(landmarks, side, state, initial_hip_y_dict, sit_start_time, squat_count):
#     global track, rep_scores, rep_grades, rep_breakdowns, avg_score, recent_rep_feedback
#     ids = SIDE_IDX[side]
#     hip_y = landmarks[ids["HIP"]].y
#     knee_y = landmarks[ids["KNEE"]].y
#     shoulder_y = landmarks[ids["SHOULDER"]].y

#     # 1) hip_init 확보 (없으면 즉시 채움) -------------- CHANGED
#     if track.get("hip_init") is None:
#         if initial_hip_y_dict[side] is None:
#             initial_hip_y_dict[side] = hip_y
#         track["hip_init"] = initial_hip_y_dict[side]
#         # print(f"[INIT] hip_init={track['hip_init']:.3f}")
#     init_hip = track["hip_init"]

#     visible_ok, vis_map = joints_visible(landmarks, side)
#     if not visible_ok:
#         feedback = "Low visibility"
#         extra = [f"{k}:{vis_map[k]:.2f}" for k in vis_map]
#         return state, squat_count, feedback, sit_start_time, initial_hip_y_dict, extra

#     # 속도 히스토리
#     track["hip_history"].append(hip_y)
#     if len(track["hip_history"]) > VEL_WINDOW:
#         track["hip_history"].pop(0)
#     if len(track["hip_history"]) >= 2:
#         avg_vel = (track["hip_history"][-1]-track["hip_history"][0])/(len(track["hip_history"])-1)
#         inst_vel = hip_y - track["hip_history"][-2]
#     else:
#         avg_vel = 0.0
#         inst_vel = 0.0

#     back_straightness = abs(hip_y - shoulder_y)
#     feedback = ""
#     extra = []

#     # START
#     if state == "START":
#         feedback = "Check posture..."
#         extra.append("Back: Good" if back_straightness < BACK_STRAIGHTNESS_THRESHOLD else "Back: Straighten!")
#         if abs(hip_y - knee_y) < ENTER_SIT_GAP:
#             state = "SIT"
#             sit_start_time = time.time()
#             track.update({
#                 "hip_init": track["hip_init"],
#                 "hip_bottom": hip_y,
#                 "knee_bottom": knee_y,
#                 "sit_frames": 0,
#                 "bottom_locked": False,
#                 "rise_frames": 0,
#                 "hip_history": [hip_y],
#                 "knee_angle_min": None,
#                 "back_dev_max": 0.0
#             })
#             # print("START -> SIT")

#     elif state == "SIT":
#         feedback, knee_angle = check_knee_angle(landmarks, side)
#         extra.append(f"KneeAngle: {knee_angle:.1f}")

#         # 최소 무릎 각도 / 허리 편차 갱신
#         if track["knee_angle_min"] is None or knee_angle < track["knee_angle_min"]:
#             track["knee_angle_min"] = knee_angle
#         back_dev = abs(hip_y - shoulder_y)
#         if back_dev > track["back_dev_max"]:
#             track["back_dev_max"] = back_dev

#         if hip_y > track["hip_bottom"]:
#             track["hip_bottom"] = hip_y
#             track["knee_bottom"] = knee_y

#         track["sit_frames"] += 1
#         depth = track["hip_bottom"] - init_hip
#         if (not track["bottom_locked"] and
#             track["sit_frames"] >= BOTTOM_MIN_FRAMES and
#             depth >= DEPTH_MIN):
#             track["bottom_locked"] = True
#             extra.append("Bottom OK")

#         if track["bottom_locked"] and inst_vel < UP_VEL_THRESH:
#             state = "RISING"
#             track["rise_frames"] = 0
#             track["hip_history"] = [hip_y]
#             track["rising_start_hip"] = track["hip_bottom"]
#             print("SIT -> RISING")
#         elif time.time() - sit_start_time > TIME_LIMIT:
#             state = "START"
#             feedback = "Timeout. Reset."
#             print("SIT -> START (timeout)")

#     elif state == "RISING":
#         feedback, knee_angle = check_knee_angle(landmarks, side)
#         extra.append(f"KneeAngle: {knee_angle:.1f}")

#         if track["knee_angle_min"] is None or knee_angle < track["knee_angle_min"]:
#             track["knee_angle_min"] = knee_angle
#         back_dev = abs(hip_y - shoulder_y)
#         if back_dev > track["back_dev_max"]:
#             track["back_dev_max"] = back_dev

#         start_hip = track.get("rising_start_hip", track.get("hip_bottom", hip_y))
#         target_y = min(init_hip + STAND_HIP_MARGIN, knee_y - KNEE_GAP)
#         # total_needed = start_hip - target_y
#         total_needed = max(start_hip - target_y, 1e-6)
#         rising_disp = start_hip - hip_y
#         progress = max(0.0, min(1.2, rising_disp / total_needed))
#         extra.append(f"prog:{progress*100:4.0f}%")

#         # if total_needed < 1e-6:
#         #     total_needed = 1e-6
#         # rising_disp = start_hip - hip_y
#         # progress = rising_disp / total_needed
#         # progress = max(0.0, min(1.2, progress))
#         # extra.append(f"prog:{progress*100:4.0f}%")

#         stand_ready = (
#             hip_y < knee_y - KNEE_GAP and
#             hip_y <= init_hip + STAND_HIP_MARGIN and
#             (track["rise_frames"] >= UP_MIN_FRAMES or progress >= STAND_PROGRESS)
#         )
#         if stand_ready:
#             # DEBUG: 스코어 입력 필드 확인
#             print("[DEBUG SCORE INPUT]",
#                   "knee_angle_min=", track.get("knee_angle_min"),
#                   "hip_init=", track.get("hip_init"),
#                   "hip_bottom=", track.get("hip_bottom"),
#                   "knee_bottom=", track.get("knee_bottom"))

#             score, breakdown, grade = compute_rep_score_simple(track)  # <-- 기존 스코어 계산 줄
#             print(f"[REP] #{squat_count+1} score={score} grade={grade} breakdown={breakdown}")
#             rep_scores.append(score); rep_grades.append(grade); rep_breakdowns.append(breakdown)
#             avg_score = round(sum(rep_scores)/len(rep_scores),1)
#             recent_rep_feedback = f"Rep {squat_count+1}: {score} ({grade})"


#             state = "STAND"
#             squat_count += 1
#             feedback = f"Squat Count: {squat_count}"
#             track.update({
#                 "hip_history": [hip_y],
#                 "rise_frames": 0,
#                 "sit_frames": 0,
#                 "bottom_locked": False,
#                 "knee_angle_min": None,
#                 "back_dev_max": 0.0,
#                 "hip_bottom": None,
#                 "knee_bottom": None
#             })

#             # track["hip_history"] = [hip_y]
#             # print("RISING -> STAND")
#             return state, squat_count, feedback, sit_start_time, initial_hip_y_dict, extra

#         if inst_vel < UP_VEL_THRESH:
#             track["rise_frames"] += 1
#             feedback = "Rising..."
#         else:
#             # 불완전 카운트
#             # squat_count += 1
#             feedback = "Incomplete"
#             extra.append("Incomplete")
#             state = "SIT"
#             sit_start_time = time.time()
#             track.update({
#                 "hip_bottom": hip_y,
#                 "knee_bottom": knee_y,
#                 "sit_frames": 0,
#                 "bottom_locked": False,
#                 "rise_frames": 0,
#                 "hip_history": [hip_y],
#                 "knee_angle_min": None,
#                 "back_dev_max": 0.0
#             })
#             print("RISING -> SIT (incomplete)")
#             return state, squat_count, feedback, sit_start_time, initial_hip_y_dict, extra
#         extra.append(f"riseF:{track['rise_frames']}")
#         extra.append(f"vel:{avg_vel:.4f}")

#     elif state == "STAND":
#         feedback = "Standing: Good posture" if back_straightness < BACK_STRAIGHTNESS_THRESHOLD else "Standing: Straighten back"
#         # if avg_vel > abs(UP_VEL_THRESH) and hip_y >= knee_y - ENTER_SIT_GAP:
#         if inst_vel > 0 and hip_y >= knee_y - ENTER_SIT_GAP:
#             state = "SIT"
#             sit_start_time = time.time()
#             track.update({
#                 "hip_bottom": hip_y,
#                 "knee_bottom": knee_y,
#                 "sit_frames": 0,
#                 "bottom_locked": False,
#                 "rise_frames": 0,
#                 "hip_history": [hip_y],
#                 "knee_angle_min": None,
#                 "back_dev_max": 0.0
#             })
#             feedback = "Descend"
#             print("STAND -> SIT")

#     return state, squat_count, feedback, sit_start_time, initial_hip_y_dict, extra

In [268]:
def process_state(landmarks, side, state, initial_hip_y_dict, sit_start_time, squat_count):
    global track, rep_scores, rep_grades, rep_breakdowns, avg_score, recent_rep_feedback
    ids = SIDE_IDX[side]
    hip_y = landmarks[ids["HIP"]].y
    knee_y = landmarks[ids["KNEE"]].y
    shoulder_y = landmarks[ids["SHOULDER"]].y

    # 1) hip_init 확보 (없으면 즉시 채움) -------------- CHANGED
    if track.get("hip_init") is None:
        if initial_hip_y_dict[side] is None:
            initial_hip_y_dict[side] = hip_y
        track["hip_init"] = initial_hip_y_dict[side]
        # print(f"[INIT] hip_init={track['hip_init']:.3f}")
    init_hip = track["hip_init"]

    visible_ok, vis_map = joints_visible(landmarks, side)
    if not visible_ok:
        feedback = "Low visibility"
        extra = [f"{k}:{vis_map[k]:.2f}" for k in vis_map]
        return state, squat_count, feedback, sit_start_time, initial_hip_y_dict, extra

    # 속도 히스토리
    track["hip_history"].append(hip_y)
    if len(track["hip_history"]) > VEL_WINDOW:
        track["hip_history"].pop(0)
    if len(track["hip_history"]) >= 2:
        avg_vel = (track["hip_history"][-1]-track["hip_history"][0])/(len(track["hip_history"])-1)
        inst_vel = hip_y - track["hip_history"][-2]
    else:
        avg_vel = 0.0
        inst_vel = 0.0

    back_straightness = abs(hip_y - shoulder_y)
    feedback = ""
    extra = []

    # START
    if state == "START":
        feedback = "Check posture..."
        extra.append("Back: Good" if back_straightness < BACK_STRAIGHTNESS_THRESHOLD else "Back: Straighten!")
        if abs(hip_y - knee_y) < ENTER_SIT_GAP:
            state = "SIT"
            sit_start_time = time.time()
            track.update({
                "hip_init": track["hip_init"],
                "hip_bottom": hip_y,
                "knee_bottom": knee_y,
                "sit_frames": 0,
                "bottom_locked": False,
                "rise_frames": 0,
                "hip_history": [hip_y],
                "knee_angle_min": None,
                "back_dev_max": 0.0,
                "slow_frames": 0
            })
            if BACK_USE_TRUNK_ANGLE and track.get("trunk_angle_baseline") is None:
                ang0 = compute_trunk_angle(landmarks, side)
                track["trunk_angle_baseline"] = ang0
                track["trunk_angle_ema"] = ang0
                track["trunk_angle_peak"] = ang0
            print("START -> SIT")

    elif state == "SIT":
        feedback, knee_angle = check_knee_angle(landmarks, side)
        extra.append(f"KneeAngle: {knee_angle:.1f}")

        # 최소 무릎 각도 / 허리 편차 갱신
        if track["knee_angle_min"] is None or knee_angle < track["knee_angle_min"]:
            track["knee_angle_min"] = knee_angle

        if BACK_USE_TRUNK_ANGLE:
            ang = compute_trunk_angle(landmarks, side)
            if track["trunk_angle_ema"] is None:
                track["trunk_angle_ema"] = ang
            else:
                track["trunk_angle_ema"] = (1 - TRUNK_EMA_ALPHA)*track["trunk_angle_ema"] + TRUNK_EMA_ALPHA*ang
            if track.get("trunk_angle_peak") is None or ang > track["trunk_angle_peak"]:
                track["trunk_angle_peak"] = ang
            # rep 최고 전경값 추적
            # peak = track.get("trunk_angle_peak", 0.0)
            # if ang > peak:
            #     track["trunk_angle_peak"] = ang
        else:
            back_dev = abs(hip_y - shoulder_y)

        # if back_dev > track["back_dev_max"]:
        #     # track["back_dev_max"] = back_dev
        #     track["back_dev_max"] = 0.8*track["back_dev_max"] + 0.2*back_dev

        if hip_y > track["hip_bottom"]:
            track["hip_bottom"] = hip_y
            track["knee_bottom"] = knee_y

        track["sit_frames"] += 1
        depth = track["hip_bottom"] - init_hip
        if (not track["bottom_locked"] and
            track["sit_frames"] >= BOTTOM_MIN_FRAMES and
            depth >= DEPTH_MIN):
            track["bottom_locked"] = True
            extra.append("Bottom OK")

        if track["bottom_locked"] and inst_vel < UP_VEL_THRESH:
            state = "RISING"
            track["rise_frames"] = 0
            track["hip_history"] = [hip_y]
            track["rising_start_hip"] = track["hip_bottom"]
            track["slow_frames"] = 0
            print("SIT -> RISING")
        elif time.time() - sit_start_time > TIME_LIMIT:
            state = "START"
            feedback = "Timeout. Reset."
            print("SIT -> START (timeout)")

    elif state == "RISING":
        feedback, knee_angle = check_knee_angle(landmarks, side)
        extra.append(f"KneeAngle: {knee_angle:.1f}")

        if track["knee_angle_min"] is None or knee_angle < track["knee_angle_min"]:
            track["knee_angle_min"] = knee_angle
        back_dev = abs(hip_y - shoulder_y)
        if back_dev > track["back_dev_max"]:
            track["back_dev_max"] = back_dev
        if BACK_USE_TRUNK_ANGLE:
            ang = compute_trunk_angle(landmarks, side)
            if track.get("trunk_angle_ema") is None:
                track["trunk_angle_ema"] = ang
            else:
                track["trunk_angle_ema"] = (1-TRUNK_EMA_ALPHA)*track["trunk_angle_ema"] + TRUNK_EMA_ALPHA*ang
            if track.get("trunk_angle_peak") is None or ang > track["trunk_angle_peak"]:
                track["trunk_angle_peak"] = ang

        start_hip = track.get("rising_start_hip", track.get("hip_bottom", hip_y))
        target_y = min(init_hip + STAND_HIP_MARGIN, knee_y - KNEE_GAP)
        total_needed = start_hip - target_y
        if total_needed < 1e-6:
            total_needed = 1e-6
        rising_disp = start_hip - hip_y
        progress = rising_disp / total_needed
        progress = max(0.0, min(1.2, progress))
        extra.append(f"prog:{progress*100:4.0f}%")

        stand_ready = (
            hip_y < knee_y - KNEE_GAP and
            hip_y <= init_hip + STAND_HIP_MARGIN
            # and (track["rise_frames"] >= UP_MIN_FRAMES or progress >= STAND_PROGRESS)
        )
        if stand_ready:
            # DEBUG: 스코어 입력 필드 확인
            print("[DEBUG SCORE INPUT]",
                  "knee_angle_min=", track.get("knee_angle_min"),
                  "hip_init=", track.get("hip_init"),
                  "hip_bottom=", track.get("hip_bottom"),
                  "knee_bottom=", track.get("knee_bottom"))

            state = "STAND"
            squat_count += 1
            feedback = f"Squat Count: {squat_count}"

            # --- 반복 점수 계산 ---
            score, breakdown, grade = compute_rep_score_simple(track)

            rep_scores.append(score)
            rep_grades.append(grade)
            rep_breakdowns.append(breakdown)
            # global avg_score, recent_rep_feedback
            avg_score = round(sum(rep_scores)/len(rep_scores),1)
            recent_rep_feedback = f"count {squat_count}: {score} ({grade})"

            print(f"[REP] #{squat_count} score={score} grade={grade} breakdown={breakdown}")

            track.update({
                "hip_history": [hip_y],
                "rise_frames": 0,
                "sit_frames": 0,
                "bottom_locked": False,
                "knee_angle_min": None,
                "back_dev_max": 0.0,
                "hip_bottom": None,
                "knee_bottom": None,
                "slow_frames": 0,
                "trunk_angle_peak": track.get("trunk_angle_ema")
            })

            # # 다음 rep 준비 변수 리셋
            # track["knee_angle_min"] = None
            # track["back_dev_max"] = 0.0
            # track["sit_frames"] = 0
            # track["bottom_locked"] = False
            # track["rise_frames"] = 0
            # track["hip_history"] = [hip_y]
            # print("RISING -> STAND")

            print("RISING -> STAND")
            return state, squat_count, feedback, sit_start_time, initial_hip_y_dict, extra
        # else:
        #     a = hip_y < knee_y - KNEE_GAP
        #     b = hip_y <= init_hip + STAND_HIP_MARGIN
        #     c = (track["rise_frames"] >= UP_MIN_FRAMES or progress >= STAND_PROGRESS)
        #     print(f"RISING conditions: {a}, {b}, {c}")
        if inst_vel < UP_VEL_THRESH:
            track["rise_frames"] += 1
            feedback = "Rising..."
        else:
            # 감속 허용: progress < 0.3 인데도 느리면 카운트다운
            if progress < 0.30:
                track.setdefault("slow_frames",0)
                track["slow_frames"] += 1
                if track["slow_frames"] >= 12:  # 12프레임 지속 시 실패
                    feedback = "Incomplete"
                    extra.append("Incomplete")
                    state = "SIT"
                    sit_start_time = time.time()
                    track.update({
                        "hip_bottom": hip_y,
                        "knee_bottom": knee_y,
                        "sit_frames": 0,
                        "bottom_locked": False,
                        "rise_frames": 0,
                        "hip_history": [hip_y],
                        "knee_angle_min": None,
                        "back_dev_max": 0.0,
                        "slow_frames": 0
                    })
                    return state, squat_count, feedback, sit_start_time, initial_hip_y_dict, extra
            else:
                track["slow_frames"] = 0
            feedback = "Rising (slow)"

        extra.append(f"riseF:{track['rise_frames']}")
        extra.append(f"ivel:{inst_vel:.4f}")
        extra.append(f"slowF:{track.get('slow_frames',0)}")
        #     # 불완전 카운트 (원치 않으면 이 블록 제거)
        #     # squat_count += 1
        #     feedback = f"Squat Count: {squat_count}"
        #     extra.append("Incomplete")
        #     state = "SIT"
        #     sit_start_time = time.time()
        #     track.update({
        #         "hip_bottom": hip_y,
        #         "knee_bottom": knee_y,
        #         "sit_frames": 0,
        #         "bottom_locked": False,
        #         "rise_frames": 0,
        #         "hip_history": [hip_y],
        #         "knee_angle_min": None,
        #         "back_dev_max": 0.0
        #     })
        #     print("RISING -> SIT (incomplete)")
        #     return state, squat_count, feedback, sit_start_time, initial_hip_y_dict, extra
        # extra.append(f"riseF:{track['rise_frames']}")
        # extra.append(f"vel:{avg_vel:.4f}")

    elif state == "STAND":
        feedback = "Standing: Good posture" if back_straightness < BACK_STRAIGHTNESS_THRESHOLD else "Standing: Straighten back"
        # if avg_vel > abs(UP_VEL_THRESH) and hip_y >= knee_y - ENTER_SIT_GAP:
        if inst_vel > 0 and hip_y >= knee_y - ENTER_SIT_GAP:
            state = "SIT"
            sit_start_time = time.time()
            track.update({
                "hip_bottom": hip_y,
                "knee_bottom": knee_y,
                "sit_frames": 0,
                "bottom_locked": False,
                "rise_frames": 0,
                "hip_history": [hip_y],
                "knee_angle_min": None,
                "back_dev_max": 0.0,
                "slow_frames": 0
            })
            feedback = "Descend"
            print("STAND -> SIT")

    return state, squat_count, feedback, sit_start_time, initial_hip_y_dict, extra

In [ ]:
cap = cv2.VideoCapture(CAM_INDEX, cv2.CAP_DSHOW if USE_DIRECTSHOW else 0)
# cap = cv2.VideoCapture(squat_videoPath)

# video_fps = cap.get(cv2.CAP_PROP_FPS)
# if not video_fps or video_fps <= 1:
#     video_fps = 30.0
# frame_duration = 1.0 / video_fps
# next_pts = time.time()

video_fps = cap.get(cv2.CAP_PROP_FPS)
if not video_fps or video_fps <= 1:
    video_fps = 30.0
base_frame_duration = 1.0 / video_fps
frame_duration = base_frame_duration / 0.5
next_pts = time.time()

# 전역 변수 초기화
squat_count = 0
initial_hip_y_dict = {"left": None, "right": None}
state = "START"
sit_start_time = None

active_side = None
side_stability_counter = 0

# FPS 측정용
prev_time = time.time()
frame_count = 0
fps = 0.0

# 세션 변수
session_running = False
session_start_time = None
last_display_frame = None

# 시작 관련
training_active = False
ready_frames = 0
countdown_start = None #  카운트다운 시작 시간

# squat goal
target_reached = False
stand_hold = 0

with mp_pose.Pose(
    model_complexity=0,  # 0: lite, 1: full, 2: heavy
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as pose:
  while cap.isOpened():
    # ====== (A) 프레임 동기 & 지연 시 드롭 ======
    now_sync = time.time()
    # 처리 지연이 2프레임 이상 누적되면 드롭
    drop_count = 0
    while now_sync > next_pts + 2*frame_duration:
        grabbed = cap.grab()  # 프레임만 건너뜀 (decode 생략)
        if not grabbed:
            break
        next_pts += frame_duration
        drop_count += 1
    # if drop_count:
    #     # 필요 시 디버그 출력:
    #     # print(f"[SYNC] dropped {drop_count} frames to catch up")
    #     pass

    ret, frame = cap.read()
    if not ret:
      print("Frame grab failed.")
      break
    next_pts += frame_duration

    # (선택) 입력 다운스케일 (고해상도일 때)
    if frame.shape[1] > 960:
        scale = 960 / frame.shape[1]
        frame = cv2.resize(frame, (960, int(frame.shape[0]*scale)))


    if not session_running and last_display_frame is not None and state == "FINISHED":
      freeze = last_display_frame.copy()
      y0 = 60
      cv2.putText(freeze, f"Completed {TARGET_REPS} Reps!", (30, y0), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,0,255), 2, cv2.LINE_AA)
      y0 += 45
      if rep_scores:
            cv2.putText(freeze, f"Avg Score: {avg_score}", (30, y0), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0,255,255), 2, cv2.LINE_AA)
            y0 += 40
            n = len(rep_breakdowns)
            avg_knee  = sum(b['knee%']  for b in rep_breakdowns)/n
            avg_back  = sum(b['back%']  for b in rep_breakdowns)/n
            avg_depth = sum(b['depth%'] for b in rep_breakdowns)/n
            avg_ctrl  = sum(b['ctrl%']  for b in rep_breakdowns)/n
            cv2.putText(freeze, f"Knee Angle: {classify_grade(avg_knee)} ({avg_knee:.1f}%)", (30, y0), cv2.FONT_HERSHEY_SIMPLEX, 0.62, (255,255,255), 2, cv2.LINE_AA)
            y0 += 28
            cv2.putText(freeze, f"Trunk Alignment: {classify_grade(avg_back)} ({avg_back:.1f}%)", (30, y0), cv2.FONT_HERSHEY_SIMPLEX, 0.62, (255,255,255), 2, cv2.LINE_AA); y0 += 28
            cv2.putText(freeze, f"Hip Depth: {classify_grade(avg_depth)} ({avg_depth:.1f}%)", (30, y0), cv2.FONT_HERSHEY_SIMPLEX, 0.62, (255,255,255), 2, cv2.LINE_AA); y0 += 28
            cv2.putText(freeze, f"Bottom Control: {classify_grade(avg_ctrl)} ({avg_ctrl:.1f}%)", (30, y0), cv2.FONT_HERSHEY_SIMPLEX, 0.62, (255,255,255), 2, cv2.LINE_AA); y0 += 38

            # 각 반복 점수/등급 라인별 출력
            for i, (s, g) in enumerate(zip(rep_scores, rep_grades), start=1):
                cv2.putText(freeze, f"Time {i}: {s} ({g})", (30, y0), cv2.FONT_HERSHEY_SIMPLEX, 0.65, (255,255,255), 2, cv2.LINE_AA)
                y0 += 26
      # cv2.putText(freeze, "s : restart, q : quit",
      #             (30, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,255), 2, cv2.LINE_AA)
      cv2.imshow("MediaPipe Pose", resize_image(freeze, MAX_WIDTH))

      key = cv2.waitKey(1) & 0xFF
      if key in (27, ord('q')):
        break
      if key == ord('s'):
        squat_count = 0
        target_reached = False
        stand_hold = 0
        state = "START"
        session_running = True
        session_start_time = time.time()
        last_display_frame = None
        track.update({"hip_init":None,"hip_bottom":None,"knee_bottom":None,
                          "sit_frames":0,"bottom_locked":False,"rise_frames":0,
                          "hip_history":[], "knee_angle_min":None,"back_dev_max":0.0})
        # 이전 기록 지우고 새로 시작하려면 아래 해제
        rep_scores.clear(); rep_grades.clear(); rep_breakdowns.clear()
        # global avg_score, recent_rep_feedback
        stand_hold = 0
        print("[Session] RESTART")
      continue

    ret, frame = cap.read()
    if not ret:
      print("Frame grab failed.")
      break

    # BGR → RGB 변환
    frame.flags.writeable = False
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(rgb)
    feedback = ""  # 피드백 초기화
    extra_lines = []

    if session_running and results.pose_landmarks:
      landmarks = results.pose_landmarks.landmark
      active_side, side_stability_counter, left_sum, right_sum = check_side(
        landmarks, active_side, side_stability_counter
      )

      if active_side is not None:
        state, squat_count, feedback, sit_start_time, initial_hip_y_dict, extra_lines = process_state(landmarks, active_side, state, initial_hip_y_dict, sit_start_time, squat_count)

      if squat_count >= TARGET_REPS:
        if not target_reached:
          target_reached = True
          stand_hold = 0
        if state == "STAND":
          stand_hold += 1
        else:
          stand_hold = 0
        extra_lines.append(f"FinishHold: {stand_hold}/{TARGET_REP_HOLD_FRAMES}")
        feedback = f"Target Reps ({TARGET_REPS}) achieved!"
        if stand_hold >= TARGET_REP_HOLD_FRAMES:
          session_running = False
          state = "FINISHED"
          last_display_frame = frame.copy()
    elif not session_running:
      feedback = "stopped"

    frame.flags.writeable = True
    frame = cv2.cvtColor(rgb, cv2.COLOR_RGB2BGR)
    if results.pose_landmarks:
            mp_drawing.draw_landmarks(
                frame,
                results.pose_landmarks,
                mp_pose.POSE_CONNECTIONS,
                landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style()
    )

    # 피드백 및 상태 정보 표시
    draw_feedback(frame, feedback, state, squat_count, extra_lines)
    # if session_running and results.pose_landmarks and active_side and 'track' in globals():
    #     draw_stand_guides(frame, track, active_side, results.pose_landmarks.landmark)
    # if recent_rep_feedback:
    #   put_text_with_bg(frame, recent_rep_feedback, (30, frame.shape[0]-150), (255,255,255), scale=0.7)
    #   put_text_with_bg(frame, f"Avg Score: {avg_score}", (30, frame.shape[0]-115), (0,255,255), scale=0.65)

    # FPS 계산 (0.5초마다 갱신)
    frame_count += 1
    now = time.time()
    if now - prev_time >= 0.5:
      fps = frame_count / (now - prev_time)
      prev_time = now
      frame_count = 0
    cv2.putText(frame, f"FPS: {fps:.1f}", (30, frame.shape[0]-20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2, cv2.LINE_AA)

    if not session_running and state != "FINISHED":
      cv2.putText(frame, f"s : start, goal : {TARGET_REPS}", (30, frame.shape[0]-55), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (30, 180, 255), 2, cv2.LINE_AA)
    else:
      cv2.putText(frame, "e : quit",
                  (30, frame.shape[0]-55),
                  cv2.FONT_HERSHEY_SIMPLEX, 0.6, (30, 180, 255), 2, cv2.LINE_AA)

    # 영상 크기 조정 (원본 비율 유지, 최대 너비 720px로 제한)
    resized_image = resize_image(frame, MAX_WIDTH)

    if session_running:
      last_display_frame = frame.copy()

    # Flip 제거 (원본 영상 그대로 표시)
    cv2.imshow('MediaPipe Pose', resized_image)
    remain = next_pts - time.time()
    if remain > 0.002:
      time.sleep(min(remain, 0.04))

    key = cv2.waitKey(1) & 0xFF
    if key in (27, ord('q')):
      break
    if key == ord('s') and not session_running:
      session_running = True
      session_start_time = time.time()
    if key == ord('e') and session_running:
      session_running = False
      last_display_frame = frame.copy()

cap.release()
cv2.destroyAllWindows()

START -> SIT
SIT -> RISING
[DEBUG SCORE INPUT] knee_angle_min= 56.84342210307057 hip_init= 0.5515562891960144 hip_bottom= 0.7041541337966919 knee_bottom= 0.7363576889038086
[REP] #1 score=70.6 grade=Good breakdown={'knee%': 100, 'back%': 0.0, 'depth%': 82.6, 'ctrl%': 100, 'knee_angle': 56.8, 'depth_ratio': 0.83, 'back_dev': 32.28, 'sit_frames': 7}
RISING -> STAND
STAND -> SIT
SIT -> RISING
[DEBUG SCORE INPUT] knee_angle_min= 65.31650995663256 hip_init= 0.5515562891960144 hip_bottom= 0.7494614720344543 knee_bottom= 0.7769917249679565
[REP] #2 score=71.9 grade=Good breakdown={'knee%': 100, 'back%': 0.0, 'depth%': 87.8, 'ctrl%': 100, 'knee_angle': 65.3, 'depth_ratio': 0.88, 'back_dev': 56.15, 'sit_frames': 12}
RISING -> STAND
STAND -> SIT
SIT -> RISING
[DEBUG SCORE INPUT] knee_angle_min= 66.68900531191596 hip_init= 0.5515562891960144 hip_bottom= 0.7142724990844727 knee_bottom= 0.7636206150054932
[REP] #3 score=66.7 grade=Fair breakdown={'knee%': 100, 'back%': 0.0, 'depth%': 76.7, 'ctrl%':